# **Add dataset**
download from kaggle directly

In [132]:
from google.colab import drive
drive.mount("/content/gdrive")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [133]:
!sudo cp -R /content/gdrive/MyDrive/kaggle/kaggle.json /content
# !ls 

In [134]:
!sudo rm -r /content/kaggle
!sudo mkdir kaggle
!sudo mv ./kaggle.json kaggle
!sudo chmod 600 kaggle/kaggle.json
!sudo kaggle datasets download -d mlg-ulb/creditcardfraud 





Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [135]:
!unzip creditcardfraud.zip

Archive:  creditcardfraud.zip
replace creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: creditcard.csv          


In [136]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score  # to check accuracy of our model
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

In [137]:
# load dataset to panda dataframe
credit_card_data=pd.read_csv('/content/creditcard.csv')

In [138]:
# first 5 rows of dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [139]:
#check data by printing last 5 rows of data
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [140]:
#dataset informations
credit_card_data.info()
# class column 0-> legit 1-> fraud

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Datatset is unbalanced.







0--> normal transaction


1--> fraud transaction

In [141]:
#distribution of legit and fraud 
credit_card_data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [142]:
# checking the number of missing values in each ccolumn
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [143]:
# it states we dont have any missing values 
#  now separate data as legit an dfraud 
legit= credit_card_data[credit_card_data.Class==0]
fraud= credit_card_data[credit_card_data.Class==1]
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [144]:
#statistical measure
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [145]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [146]:
#compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


#### Gave mean of normal and fraud transactions

#Under -Sampling


build a sample dataset containing similar distribution of normal and fraud transactions

no. of fraud transactions ----> 492 so to make even take 492 normal transactions and join with fraud transactions

In [147]:
n=492
legit_sample=legit.sample(n)

conactenate dataframes


In [148]:
new_dataset=pd.concat([legit_sample,fraud],axis=0)
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
107827,70636.0,-0.613390,0.917564,1.870088,0.597198,0.025759,0.189081,0.512747,0.179412,-0.405712,...,0.092669,0.257875,-0.008728,0.056644,-0.260381,-0.492945,-0.004928,0.061438,29.90,0
42244,41012.0,-2.241933,-5.287394,-2.147628,2.583820,-1.728497,-0.530352,2.993815,-0.849004,-0.547925,...,1.110897,-0.865194,-1.755071,0.509848,0.195926,-0.491947,-0.349761,0.328911,1823.20,0
24234,33132.0,-0.618786,0.893818,1.536855,0.591117,0.991298,0.819718,0.434979,0.264172,-1.081991,...,-0.142399,-0.463140,-0.263855,-1.173770,0.236460,-0.367500,0.098646,0.066922,9.99,0
149618,91589.0,-0.904939,0.156087,1.965470,0.053286,0.765608,0.666300,-0.381386,0.351493,1.355169,...,0.111781,0.505171,-0.389765,0.185982,0.573324,0.033886,0.045239,0.069657,12.99,0
163885,116278.0,-1.133124,-0.322168,0.133066,-2.557030,1.794994,-0.356273,0.434665,-0.038204,-1.597729,...,0.158886,0.249745,-0.717081,-0.307219,1.287669,0.002106,-0.181345,0.064347,10.00,0


##AXIS 
0---> row<br>
1--> columns

In [149]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [150]:
new_dataset['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [151]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95389.197154,-0.129353,0.068509,-0.077015,0.056315,0.060694,0.018655,-0.008416,-0.086138,-0.037107,...,0.008750,0.019090,-0.055492,-0.054081,-0.017293,0.007862,0.012815,0.009570,0.005059,95.289797
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


now as we got the sample dataset  we need to train the model so hence we are splitting data into target and features

<br><br> here our target is Class as it tells us about legitimacy of transaction

In [152]:
X=new_dataset.drop(columns='Class',axis=1)
Y=new_dataset['Class']

In [153]:
print(Y)

107827    0
42244     0
24234     0
149618    0
163885    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [154]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
107827   70636.0 -0.613390  0.917564  1.870088  0.597198  0.025759  0.189081   
42244    41012.0 -2.241933 -5.287394 -2.147628  2.583820 -1.728497 -0.530352   
24234    33132.0 -0.618786  0.893818  1.536855  0.591117  0.991298  0.819718   
149618   91589.0 -0.904939  0.156087  1.965470  0.053286  0.765608  0.666300   
163885  116278.0 -1.133124 -0.322168  0.133066 -2.557030  1.794994 -0.356273   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [155]:
X.Amount.describe()

count     984.000000
mean      108.750559
std       264.320488
min         0.000000
25%         1.620000
50%        18.105000
75%        99.862500
max      3075.000000
Name: Amount, dtype: float64

## now we need to split data into training and testing data for model

In [156]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,stratify=Y,random_state=2)

stratify<br>returns training and test subsets that have the same proportions of class labels as the input dataset.

# Model Training
logistic regresion

In [157]:
model=LogisticRegression()

In [158]:
model = make_pipeline(preprocessing.StandardScaler(), LogisticRegression())
model.fit(X_train,Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

# Model Evaluation

In [159]:
#accuracy
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [160]:
print('Accuracy on training data: ',training_data_accuracy)

Accuracy on training data:  0.9498644986449865


In [161]:
#accuracy on test data
X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print('Accuracy on testing data: ',testing_data_accuracy)

Accuracy on testing data:  0.9308943089430894
